# MSTICPy
**MSTICPy** (pronounced miss-tick-pie) is a Python package of CyberSecurity tools for data retrieval, analysis, enrichment and visualization.

In [1]:
import os, msticpy
print("MSTICPy:", msticpy.__version__)
print("MSTICPYCONFIG:", os.getenv("MSTICPYCONFIG"))

MSTICPy: 2.16.2
MSTICPYCONFIG: C:\Users\ronal\Desktop\JupyterNotebook\msticpyconfig.yaml


In [5]:
from msticpy.context.tilookup import TILookup

ti = TILookup()
print("Loaded providers:", list(ti.loaded_providers.keys()))

Loaded providers: ['VirusTotal', 'OTX']


In [ ]:
# do a test lookup of a hash IOC
ioc = "ed01ebfbc9eb5bbea545af4d01bf5f1071661840480439c6e5babe8e080e41aa"
res = ti.lookup_ioc(ioc, providers=["VirusTotal", "OTX"], ioc_type="hash")

try:
    df = TILookup.result_to_df(res)
except AttributeError:
    df = ti.result_to_df(res)

df.head()

Observables processed:   0%|          | 0/2 [00:00<?, ?obs/s]

,QuerySubtype,Result,Details,RawResult,Reference,Status,Ioc,IocType,SafeIoc,Severity,Provider


In [13]:
# test Azure Sentinel connection
from msticpy.data import QueryProvider
from msticpy.common.wsconfig import WorkspaceConfig

ws = WorkspaceConfig()              # read default from config
qry = QueryProvider("MSSentinel")   # Sentinel / Log analytics
qry.connect(workspace="Default", auth_types=["cli","devicecode"])           # use Azure CLI token / default creds

df_test = qry.exec_query("print x=1")
df_test

connected


,x
0,1


In [21]:
#3.1 Verify Microsoft Sentinel settings
import msticpy
from msticpy.config import MpConfigFile, MpConfigEdit
import os
import json
from pathlib import Path

mp_conf = "msticpyconfig.yaml"

mp_env = os.environ.get("MSTICPYCONFIG")
mp_conf = mp_env if mp_env and Path(mp_env).is_file() else mp_conf

if not Path(mp_conf).is_file():
    print(
        "No msticpyconfig.yaml was found!",
        "Please check that there is a config.json file in your workspace folder.",
        "If this is not there, go back to the Microsoft Sentinel portal and launch",
        "this notebook from there.",
        sep="\n"
    )
else:
    mpedit = MpConfigEdit(mp_conf)
    mpconfig = MpConfigFile(mp_conf)
    ws = mpconfig.settings["AzureSentinel"]["Workspaces"]
    try:
        print(f"Configured Sentinel workspaces:", json.dumps(ws.to_dict(), indent=2))
    except Exception:
        from pprint import pprint
        pprint(ws)

msticpy.settings.refresh_config()

Label(value='Loading. Please wait.')

{'Default': {'ResourceGroup': 'msticpy',
             'SubscriptionId': '553563e5-5912-4c7e-be77-a1927bc0286d',
             'TenantId': '4e3edda0-4a9b-41ee-8197-23223ec3de4b',
             'WorkspaceId': '515d3fd2-4214-4aee-bd2b-592d6efaa59c',
             'WorkspaceName': 'MSTICpy'}}


In [22]:
#3.3 Load a QueryProvider for Microsoft Sentinel
msticpy.settings.refresh_config()

qry_prov = msticpy.QueryProvider("AzureSentinel")

In [ ]:
#3.5 test connection using a MSTICPy built-in Microsoft Sentinel query
from IPython.display import display, Markdown
def md(text: str):
    display(Markdown(text))

alerts_df = qry_prov.SecurityAlert.list_alerts(start=qry_prov.query_time.start)

if alerts_df.empty:
    md("The query returned no rows for this time range. You might want to increase the time range.")

alerts_df.head()

The query returned no rows for this time range. You might want to increase the time range.

,TenantId,TimeGenerated,AlertDisplayName,AlertName,Severity,Description,ProviderName,VendorName,VendorOriginalId,SystemAlertId,ResourceId,SourceComputerId,AlertType,ConfidenceLevel,ConfidenceScore,IsIncident,StartTimeUtc,EndTimeUtc,ProcessingEndTime,RemediationSteps,ExtendedProperties,Entities,SourceSystem,WorkspaceSubscriptionId,WorkspaceResourceGroup,ExtendedLinks,ProductName,ProductComponentName,AlertLink,Status,CompromisedEntity,Tactics,Techniques,SubTechniques,Type


In [29]:
#4.2 configure and test Virus Total
mpe = msticpy.MpConfigEdit()
mpe

Label(value='Loading. Please wait.')

In [33]:
display(mpedit)
mpedit.set_tab("TI Providers")

In [34]:
display(mpedit)
mpedit.set_tab("GeoIP Providers")

In [35]:
#4.3 test VirusTotal Lookup
msticpy.settings.refresh_config()

ti = msticpy.TILookup()

ti_resp = ti.lookup_ioc("85.214.149.236", providers=["VirusTotal"])

ti_df = ti.result_to_df(ti_resp)
ti.browse_results(ti_df, severities="all")

Observables processed:   0%|          | 0/1 [00:00<?, ?obs/s]

In [43]:
#4.4 test IP geolocation lookup with Maxmind GeoLite2
msticpy.settings.refresh_config()

geo_ip = msticpy.GeoLiteLookup()
raw_res, ip_entity = geo_ip.lookup_ip("85.214.149.236")
display(ip_entity[0])

C:\Users\ronal\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\msticpy\common\provider_settings.py:335: UserWarning: Environment variable account_id (OtherProviders.GeoIPLite.Args.AuthKey) was not set
  warnings.warn(


IpAddress(Address=85.214.149.236, Location={ 'CountryCode': 'DE',
  'CountryOrRegionName':...)